# The preceeding ipynb is semi more general in selecting it's blurring region.
# It is capable of selecting cuboid, i.e. unequal sides
# This ipynb restricts to only cubes, i.e. equal sides, as an attempt to be more computationally efficient
# The reason for this restriction relies on the assumption that the blurring region are spheres

## Importing Libraries

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import py21cmfast as p21c
# import caffeine
from datetime import datetime
import time
import logging, sys, os

## Daily Python Zen

In [ ]:
import this

## Set logger to log caching activity

In [ ]:
logger = logging.getLogger('21cmFAST')
logger.setLevel(logging.INFO)

## Version of 21cmFAST

In [ ]:
print(f"Using 21cmFAST version {p21c.__version__}")

## Number of cores running

In [ ]:
print(f'Number of threads running = {os.cpu_count()}')

## Reset cache location 

In [ ]:
p21c.config['direc'] = '/lustre/aoc/projects/hera/wchin/21cmFAST-cache'

## Gaussian function

In [ ]:
def Gaussian(x, sigma):  # μ=0, σ=1/sqrt(2), π=1
    Gaussian = np.exp(-(x/sigma)**2)
    return Gaussian

## Measure the distance of each voxel to the center

In [ ]:
def distance_from_coordinate(box_length):
        
    index = np.arange(-0.5*(box_length-1), 0.5*(box_length+1))

    x_mesh, y_mesh, z_mesh = np.meshgrid(index, index, index, indexing='ij')
    
    distance = np.sqrt((x_mesh)**2 + (y_mesh)**2 + (z_mesh)**2)
    
    return distance

## Random Coordinate

In [ ]:
def random_voxels(box_length, number_of_coordinates):
    
    np.random.seed()  # set seed to a randome number
#     np.random.seed(4) ; np.random.rand(10)  # specifying seed for testing purposes

    coordinates = np.random.randint(0, box_length, size=(number_of_coordinates, 3))

    return coordinates

## Select a Smaller Cube with Sides 2R+1 Voxels, Centered about the Random Coordinate

In [ ]:
def cube_centered_about_coordinate(coordinate_array, radius, box):
    x_ind1 = int(coordinate_array[0]-radius)  # x-domian
    x_ind2 = int(coordinate_array[0]+radius+1)  # ending index is not inclusive
    y_ind1 = int(coordinate_array[1]-radius)  # y-domain
    y_ind2 = int(coordinate_array[1]+radius+1)
    z_ind1 = int(coordinate_array[2]-radius)  # z-domain
    z_ind2 = int(coordinate_array[2]+radius+1)
        
    if x_ind1 < 0:  # periodic boundary conditions
        x_inds = np.r_[(x_ind1+len(box)):len(box), 0:x_ind2]
    elif x_ind2 > len(box):
        x_inds = np.r_[x_ind1:len(box), 0:(x_ind2-len(box))]
    else:
        x_inds = np.r_[x_ind1:x_ind2]

    if y_ind1 < 0:
        y_inds = np.r_[(y_ind1+len(box)):len(box), 0:y_ind2]
    elif y_ind2 > len(box):
        y_inds = np.r_[y_ind1:len(box), 0:(y_ind2-len(box))]
    else:
        y_inds = np.r_[y_ind1:y_ind2]

    if z_ind1 < 0:
        z_inds = np.r_[(z_ind1+len(box)):len(box), 0:z_ind2]
    elif z_ind2 > len(box):
        z_inds = np.r_[z_ind1:len(box), 0:(z_ind2-len(box))]
    else:
        z_inds = np.r_[z_ind1:z_ind2]
            
    indices = np.ix_(x_inds, y_inds, z_inds)
        
    try:
        output_box = box[indices]
        
    except IndexError:  # sample region larger than box.
        print(f'coordinate array = {coordinate_array}')
        print(f'radius = {radius}')
        print(f'box length = {len(box)}')
        print(f'x_ind1 = {x_ind1}')
        print(f'x_ind2 = {x_ind2}')
        print(f'x_inds = {x_inds}')
        print(f'x_ind1 = {y_ind1}')
        print(f'x_ind2 = {y_ind2}')
        print(f'y_inds = {y_inds}')
        print(f'x_ind1 = {z_ind1}')
        print(f'x_ind2 = {z_ind2}')
        print(f'z_inds = {z_inds}')
        
    return output_box

## Gausssian Sphere Averaging

In [ ]:
def gaussian_sphere_average(distance_box, radius, input_box, shell_num, sigma_factor):
    
    mean = np.zeros(shell_num)

    shell_radius_edges = np.linspace(0,1,shell_num+1)
    # sigma_factor number of sigmas the weighting goes out to, sigma = radius

    
    
# ======================================================================================================================
    # Radius Ratio 1
    shell_center = 0.5*(shell_radius_edges[1:] + shell_radius_edges[:-1])*sigma_factor 
    weight = Gaussian(x=shell_center, sigma=1)
    
    # Radius Ratio 2
#     shell_center = 0.5*(shell_radius_edges[1:] + shell_radius_edges[:-1])*sigma_factor*radius
#     weight = Gaussian(x=shell_center, sigma=radius)
# ======================================================================================================================


    
    for ii in range(shell_num):
        condition = np.logical_and(
            distance_box <= shell_radius_edges[ii+1]*radius, 
            distance_box > shell_radius_edges[ii]*radius
        )
        inside_shell_mean = np.mean(input_box[condition])
        mean[ii] = inside_shell_mean
        
    Gaussian_mean = np.average(mean, weights=weight)
    
    return Gaussian_mean

## Top Hat Sphere Averaging

In [ ]:
def top_hat_sphere_average(distance_box, radius, input_box):
    
    condition = distance_box <= radius
    mean = np.mean(input_box[condition])
    
    return mean

## Top Hat Cube Averaging

In [ ]:
def top_hat_cube_average(input_box):
    
    mean = np.mean(input_box)
    
    return mean

## Sphere Blurring Function

In [ ]:
def Average_Neutral_Fraction_Distribution(
    box, 
    radius, 
    iteration, 
    shell_num=6, 
    sigma_factor=4, 
    blur_shape=None
):
    
    box = box.copy()  # make copy of input box to have a separate box
    
    mean_data = np.zeros(iteration)  # empty list for data collection

    rand_coords = random_voxels(len(box), iteration)  # iteration number of random coordinates in the box

    
    if blur_shape == 'Gaussian_sphere':
        
        
# ======================================================================================================================
        # Radius Ratio 1
#         radius = int(round(radius*1.4370397097748921*3))
        
        # Radius Ratio 2
        radius = int(round(radius*((4/3/np.sqrt(np.pi))**(1/3))*13/4))  # 13/4 --> speculated correction factor
# ======================================================================================================================
        
        
        # check to see if averaging region is largert than the box itself
        if radius > (len(box)-1)/2:
            print(f'Radius = {radius} > {(len(box)-1)/2} = (Box Length-1)/2, averaging region is larger than the box itself.')
            print('Code terminated.')
            sys.exit()
            
        dist_frm_coord_box = distance_from_coordinate(radius*2+1)
        for i in range(iteration):
            cube_region_box = cube_centered_about_coordinate(rand_coords[i, :], radius, box)
            mean = gaussian_sphere_average(dist_frm_coord_box, radius, cube_region_box, shell_num, sigma_factor)
            mean_data[i] = mean
        
    elif blur_shape == 'top_hat_sphere':
        
        # check to see if averaging region is largert than the box itself
        if radius > (len(box)-1)/2:
            print(f'Radius = {radius} > {(len(box)-1)/2} = (Box Length-1)/2, averaging region is larger than the box itself.')
            print('Code terminated.')
            sys.exit()

        dist_frm_coord_box = distance_from_coordinate(radius*2+1)
        
        for i in range(iteration):
            cube_region_box = cube_centered_about_coordinate(rand_coords[i, :], radius, box)
            mean = top_hat_sphere_average(dist_frm_coord_box, radius, cube_region_box)
            mean_data[i] = mean
            
    elif blur_shape == 'top_hat_cube':
                
        radius = int(round((radius*((4*np.pi/3)**(1/3))-1)/2))  # ratio determiend by equating the volumes of cube & sphere

        # check to see if averaging region is largert than the box itself
        if radius > (len(box)-1)/2:
            print(f'Radius = {radius} > {(len(box)-1)/2} = (Box Length-1)/2, averaging region is larger than the box itself.')
            print('Code terminated.')
            sys.exit()
        
        for i in range(iteration):
            cube_region_box = cube_centered_about_coordinate(rand_coords[i, :], radius, box)
            mean = top_hat_cube_average(cube_region_box)
            mean_data[i] = mean
    else:
        print('Blurring shape assumed to be a Gaussian sphere with 4 shells \
              weighted by equally spaced values from 0 sigma to 4 sigma.')
                
        
# ======================================================================================================================
        # Radius Ratio 1
#         radius = int(round(radius*1.4370397097748921*3))
        
        # Radius Ratio 2
        radius = int(round(radius*((4/3/np.sqrt(np.pi))**(1/3))*13/4))  # 13/4 --> speculated correction factor
# ======================================================================================================================
        
        
        # check to see if averaging region is largert than the box itself
        if radius > (len(box)-1)/2:
            print(f'Radius = {radius} > {(len(box)-1)/2} = (Box Length-1)/2, averaging region is larger than the box itself.')
            print('Code terminated.')
            sys.exit()
        
        dist_frm_coord_box = distance_from_coordinate(radius*2+1)
        
        for i in range(iteration):
            cube_region_box = cube_centered_about_coordinate(rand_coords[i, :], radius, box)
            mean = gaussian_sphere_average(dist_frm_coord_box, radius, cube_region_box, shell_num, sigma_factor)
            mean_data[i] = mean
            
    return mean_data

## Histogram Function

In [ ]:
def histogram(
    y1s, 
    figure_shape, 
    y2s=None,
    y3s=None,
    marker_lines=None,
    y1s_label=None,
    y2s_label=None,
    y3s_label=None,
    title=None, 
    shared_title=None,
    shared_title_x_position=0.5,   # figure coordinates, max=1 I think
    shared_title_y_position=0.92,
    shared_x_label=None, 
    shared_x_label_x_position=0.5,
    shared_x_label_y_position=0.08,
    shared_y_label=None, 
    shared_y_label_x_position=0.07,
    shared_y_label_y_postion=0.5,
    x_start=0, 
    x_stop=1, 
    bin_num=100, 
    color='white', 
    figure_size=(18,7), 
    font_size=15, 
    horizontal_gap=0.05, 
    vertical_gap=0.05, 
    y_scale='linear', 
    y_notation='plain', 
    share_x_axis=True, 
    share_y_axis=True,
    dpi=100
):  # a: x start, b: x stop
    
    bin_edges = np.linspace(x_start, x_stop, bin_num) # 100 bins from 0-1
    
    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
    
    if np.array(y1s).ndim > 2:  # hopefully more robust condition.
        
        fig, axes = plt.subplots(
            figure_shape[0], 
            figure_shape[1], 
            figsize=figure_size, 
            sharex=share_x_axis, 
            sharey=share_y_axis, 
            gridspec_kw={"hspace":vertical_gap, 'wspace':horizontal_gap},
            dpi=dpi
        )
        
        if shared_title != None:
            
            fig.suptitle(
                x=shared_title_x_position, 
                y=shared_title_y_position, 
                t=shared_title, ha='center', 
                size=1.5*font_size, 
                color=color
            )
            
        if shared_x_label != None:  # shared x label
            fig.text(
                x=shared_x_label_x_position, 
                y=shared_x_label_y_position, 
                s=shared_x_label, ha='center', 
                size=font_size, 
                color=color
            )

        if shared_y_label != None:  # shared y label
            fig.text(
                x=shared_y_label_x_position, 
                y=shared_y_label_y_postion, 
                s=shared_y_label, 
                va='center', 
                rotation='vertical', 
                size=font_size, 
                color=color
            )
        
        for i, y1 in enumerate(y1s):
            
            for ii, z in enumerate(redshifts):
                axes.flatten()[i].plot(
                    bin_centers, 
                    np.histogram(y1s[i,ii,:], bins=bin_edges)[0], 
                    marker_lines[ii], 
                    label=f'{y1s_label} z={z}'
                )
            
            if iterable(y2s):
                axes.flatten()[i].plot(
                    bin_centers, 
                    np.histogram(y2s[i], bins=bin_edges)[0], 
                    '--', 
                    label=y2s_label
                )
                
                if iterable(y3s):
                    axes.flatten()[i].plot(
                        bin_centers, 
                        np.histogram(y3s[i], bins=bin_edges)[0], 
                        label=y3s_label
                    )
                    
                    
            if y1s_label != None:
                axes.flatten()[i].legend()
                
            if title != None:
                axes.flatten()[i].set_title(title[i], color=color, fontsize=font_size)
                
            axes.flatten()[i].set_yscale(y_scale)
            
            if y_notation == 'sci':
                axes.flatten()[i].ticklabel_format(
                    axis='y', 
                    style=y_notation, 
                    scilimits=(0,0), 
                    useMathText=True
                )
                
            axes.flatten()[i].tick_params(
                color=color, 
                labelcolor=color, 
                labelsize=font_size, 
                size=font_size
            )  # font style

            for spine in axes.flatten()[i].spines.values():  # figure color
                spine.set_edgecolor(color)
                
    else:
        
        fig, ax = plt.subplots(figsize=figure_size)
        
        ax.plot(bin_centers, np.histogram(y1s, bins=bin_edges)[0], label=y1s_label)
        
        if y2s != None:
            ax.plot(bin_centers, np.histogram(y2s, bins=bin_edges)[0], '--', label=y2s_label)
            
            if y3s == None:
                ax.legend()
                
            else:
                ax.plot(bin_centers, np.histogram(y3s, bins=bin_edges)[0], '-.', label=y3s_label)
                ax.legend()
                
        if title != None:
            ax.set_title(title, color=color, fontsize=font_size)
            
        ax.set_yscale(y_scale)
        
        if y_notation == 'sci':
            ax.ticklabel_format(axis='y', style=y_notation, scilimits=(0,0), useMathText=True)
            
        ax.tick_params(color=color, labelcolor=color, labelsize=font_size)  # font style

        for spine in ax.spines.values():  # figure color
            spine.set_edgecolor(color)
            
    plt.show()

## Check if object is iterable

In [ ]:
def iterable(obj):
    
    try:
        iter(obj)
        
    except Exception:
        return False
    
    else:
        return True

## Cosmological Parameters (Default is used when no input is specified)

In [ ]:
cosmo_params = p21c.CosmoParams()

## User Parameters, like box length, number of voxels (i.e. resolution) etc.

In [ ]:
BOX_LEN=301  # 300, 301
HII_DIM=301  # 450, 301

user_params = p21c.UserParams(
    BOX_LEN=BOX_LEN,  # Box length in Mpc
    DIM=4*HII_DIM,      # Number of Voxels for hight resolution 
    HII_DIM=HII_DIM,  # Number of Voxels for low resolution 
    N_THREADS=os.cpu_count()
)

## Creating initial conditions box

In [ ]:
start_time = datetime.now()
print(f'Excution qued at {start_time}')

init_cond = p21c.initial_conditions(
    cosmo_params=cosmo_params,
    user_params=user_params,
)

end_time = datetime.now()
execution_time = end_time - start_time
print(f'Execution completed at {end_time}')
print(f'Execution time = {execution_time}')

## Generate ionized boxes as a function of redshift

In [ ]:
# redshifts = np.arange(8, 4.5, -0.5)
redshifts = np.array([6.5])
ionized_boxes = np.zeros((len(redshifts), HII_DIM, HII_DIM, HII_DIM))

for i, z in enumerate(redshifts):
    ionized_box = p21c.ionize_box(redshift=z, init_boxes=init_cond).xH_box
    ionized_boxes[i] = ionized_box

## Generate Average Neutral Fraction Distributions as a function of redshift

In [ ]:
def generate_distributions(
    boxes=ionized_boxes,
    radii=np.arange(8, 17, 1),
    iterations=10**2,
    sigma_factor=4,
    shell_number=6
):

    gaussians = np.zeros((len(radii), len(redshifts), iterations))
    
    
    # temporary
# ===================================================================================================================
    spheres = np.zeros((len(radii), iterations))
    cubes = np.zeros((len(radii), iterations))
# ===================================================================================================================


    start_time = datetime.now()
    current_time = start_time
    print(f'Progress = 0%, localtime = {start_time}')

    for i, radius in enumerate(radii):
        
    # temporary
# ===================================================================================================================        
        sphere = Average_Neutral_Fraction_Distribution(
                box=boxes[0],
                radius=radius,
                iteration=iterations,
                blur_shape='top_hat_sphere'
        )

        cube = Average_Neutral_Fraction_Distribution(
                box=boxes[0],
                radius=radius,
                iteration=iterations,
                blur_shape='top_hat_cube'
        )
        
        spheres[i, :] = sphere
        cubes[i, :] = cube
# ===================================================================================================================
        
        for ii, box in enumerate(boxes):
                        
            gaussian = Average_Neutral_Fraction_Distribution(
                box=box,
                radius=radius,
                iteration=iterations,
                sigma_factor=sigma_factor,
                shell_num=shell_number,
                blur_shape='Gaussian_sphere'
            )

            gaussians[i, ii, :] = gaussian


        # print progress and local time
        previous_time = current_time
        current_time = datetime.now()
        loop_time = current_time - previous_time
        elapsed_time = current_time - start_time
        print(f'progress = {int(round((i+1)*100/len(radii)))}%, localtime =\
 {current_time}, loopexecuted in {loop_time}, elapsedtime = {elapsed_time}')
        
    # temporary
# =================================================================================================================== 
    return gaussians, spheres, cubes
# =================================================================================================================== 
    
#     return gaussians

## Generate the distributions and storing the data in global variables

In [ ]:
radii = np.arange(34, 3, -2)  # array([34, 32, 30,... 6, 4]) units: voxels (51, 3, -3)
iterations = int(1.5*10**4)

    # temporary
# =================================================================================================================== 
gaussians, spheres, cubes = generate_distributions(
# =================================================================================================================== 

    
# gaussians = generate_distributions(
    boxes=ionized_boxes,
    radii=radii,
    iterations=iterations
)

## Creating Histogram

In [ ]:
histogram(
    y1s=gaussians, 
    
    # temporary
# =================================================================================================================== 
    y2s=spheres,
    y3s=cubes,
# =================================================================================================================== 
    
    marker_lines=['-']*len(redshifts),
    y1s_label='Gauss',
    
    # temporary
# =================================================================================================================== 
    y2s_label='Sphere',
    y3s_label='Cube',
# =================================================================================================================== 
    
    title=[f'Radius={r*BOX_LEN/HII_DIM:.2f}Mpc' for r in radii],
    shared_title=f'Distribution of Average \
Neutral Fraction (RR2, Gaussian weights of RR1, iterations={iterations:.2e})',
    shared_y_label='Counts',
    shared_x_label='Neutral Fraction',
    figure_shape=(4,4), 
    figure_size=(18,18),
    vertical_gap=0.1,
    horizontal_gap=0.25,
    y_scale='log',
#     y_notation='sci',
    share_y_axis=False
#     dpi=1000
)